In [30]:
import azureml.core
from azureml.core import Workspace, Datastore, Dataset, Experiment

mlonazure_ws = Workspace.from_config()

mlonazure_ds = mlonazure_ws.get_default_datastore()

print('Workspace Name: ' + mlonazure_ws.name, 
      'Resource Group: ' + mlonazure_ws.resource_group,
      'Default Storage Account Name: ' + mlonazure_ds.account_name,
      'AzureML Core Version: ' + azureml.core.VERSION,
      sep = '\n')

Workspace Name: mlonazure-ws
Resource Group: mlonazure-rg
Default Storage Account Name: mlonazurews7669880942
AzureML Core Version: 1.15.0


In [31]:
new_model_name = 'AutoML8c77c369671'
api_service_name = 'automl-ojsales-api'

In [ ]:
# import json
# import pickle
# import numpy as np
# import pandas as pd
# import azureml.train.automl
# import joblib
# from azureml.core.model import Model

# #input_sample = pd.DataFrame(data=[{'sepal_length': 5.9, 'sepal_width': 3.0, 'petal_length': 5.1, 'petal_width': 1.8}])

# #input_sample = pd.DataFrame(data=[{"Column1": pd.Series(["1990-06-14T00:00:00.000Z"], dtype="datetime64[ns]"), "WeekStarting": pd.Series(["1990-06-14T00:00:00.000Z"], dtype="datetime64[ns]"), "Store": pd.Series(["1000"], dtype="int64"), "Brand": pd.Series(["dominicks"], dtype="object"), "Advert": pd.Series(["1"], dtype="int64"), "Price": pd.Series(["2.59"], dtype="float64"), "Revenue": pd.Series(["31087.77"], dtype="float64")}])

# input_sample = pd.DataFrame(data=[{"Column1": pd.Series(["1990-06-14T00:00:00.000Z"], dtype="datetime64[ns]"), "WeekStarting": pd.Series(["1990-06-14T00:00:00.000Z"], dtype="datetime64[ns]"), "Store": pd.Series(["1000"], dtype="int64"), "Brand": pd.Series(["dominicks"], dtype="object"), "Advert": pd.Series(["1"], dtype="int64"), "Price": pd.Series(["2.59"], dtype="float64"), "Revenue": pd.Series(["31087.77"], dtype="float64")}])

# #best_run.download_file('outputs/model.pkl', new_model_name)

# model = Model(workspace = mlonazure_ws, name = new_model_name)

# model.download(target_dir='.',exist_ok = True)

# myModel = joblib.load(filename='model.pkl')

# # data = input_sample

# result = myModel.predict(input_sample)

# json.dumps({"result": result.tolist()})
    
# #display(result)


In [44]:

# import json
# input_sample = pd.DataFrame(data=[{"Column1": pd.Series(["1990-06-14T00:00:00.000Z"], dtype="datetime64[ns]"), "WeekStarting": pd.Series(["1990-06-14T00:00:00.000Z"], dtype="datetime64[ns]"), "Store": pd.Series(["1000"], dtype="int64"), "Brand": pd.Series(["dominicks"], dtype="object"), "Advert": pd.Series(["1"], dtype="int64"), "Price": pd.Series(["2.59"], dtype="float64"), "Revenue": pd.Series(["31087.77"], dtype="float64")}])


# test_sample = json.dumps({'data': [
#     ["1990-06-14T00:00:00.000Z","2000-06-14T00:00:00.000Z",1000, 'dominicks', 1, 2.59,3108.77]
# ]})

# api_service.run(input_data=test_sample)

'{"forecast": [NaN], "index": [{"WeekStarting": 960940800000, "Store": 1000, "Brand": "dominicks"}]}'

In [45]:
#Note that you need to run ../00. Get Data from OpenDatasets.ipynb to get and register this dataset.
datasetName_train = 'OJSales_Train_Subset'
datasetName_validate = 'OJSales_Validate_Subset'

data_train_ds = Dataset.get_by_name(mlonazure_ws,datasetName_train,version='latest')
data_validate_ds = Dataset.get_by_name(mlonazure_ws,datasetName_validate,version='latest')

In [64]:
pdValidate = data_validate_ds.to_pandas_dataframe().head(1)
#pdValidate = data_validate_ds.to_pandas_dataframe()

In [65]:
pdValidate.head(10)

,Column1,WeekStarting,Store,Brand,Quantity,Advert,Price,Revenue
0,1992-06-04,1992-06-04,1000,dominicks,11225,1,2.47,27725.75


In [66]:
inferencedata_json = pdValidate.to_json(orient='records', date_format='iso')
inferencedata_json = '{"data": ' + inferencedata_json +'}'

In [67]:
print(inferencedata_json)

{"data": [{"Column1":"1992-06-04T00:00:00.000Z","WeekStarting":"1992-06-04T00:00:00.000Z","Store":1000,"Brand":"dominicks","Quantity":11225,"Advert":1,"Price":2.47,"Revenue":27725.75}]}


In [68]:
from azureml.core.webservice import AksWebservice 

api_service = AksWebservice(workspace = mlonazure_ws, name = api_service_name)


In [69]:
forecast_results= api_service.run(input_data=inferencedata_json)

In [70]:
forecast_results

'{"forecast": [11264.858072916666], "index": [{"WeekStarting": 707616000000, "Store": 1000, "Brand": "dominicks"}]}'